#Here are present the function that are useful for the 2 first part of the Project. 
#We use it to make the code more readable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import seaborn as sns
sns.set_style("whitegrid")
import string
from spacy.lang.en.stop_words import STOP_WORDS
from pprint import pprint
import functools
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
%%capture
!pip install -U spacy
# Download the French language model
!python -m spacy download fr

In [ ]:
sp = spacy.load('fr_core_news_sm')

Function to print a classification report 

In [ ]:
def Evaluate(true, pred):
    precision = precision_score(true, pred, average='weighted')
    recall = recall_score(true, pred, average='weighted')
    f1 = f1_score(true, pred, average='weighted')
    accuracy = accuracy_score(true, pred)
    print(f"CLASSIFICATION REPORT:\n\tAccuracy: {accuracy:.4f}\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

Function to plot a confusion matrix

In [ ]:
def Accuracy_conf_mat(y_test, y_pred_test):
  conf_mat = confusion_matrix(y_test, y_pred_test)
  fig, ax = plt.subplots(figsize=(10,10))
  sns.heatmap(conf_mat, annot=True, fmt='d')
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.xaxis.set_ticklabels(['A1', 'A2','B1','B2','C1','C2']); ax.yaxis.set_ticklabels(['A1', 'A2','B1','B2','C1','C2']);
  plt.show()

Function to print a sample of sentences that are not correctly classified

In [ ]:
def Sentences_badly_classified(a,b, X_test, prediction, y_test):
  classification_sample = list(enumerate(zip(X_test, prediction, y_test)))
  classification_sample = classification_sample[a:b] #take a sample
  for (row_index, (input, prediction, label)) in classification_sample :
    if prediction != label:
      print((input, 'has been classified as ', prediction, 'and should be ', label))

Function to return a prediction

In [ ]:
def Prediction(model, test):
  prediction = model.predict(test)
  return prediction

Function to return the DataFrame in the correct form for submission




In [ ]:
def submission(id, prediction):
  d = {'id': id, 'difficulty': prediction}
  df = pd.DataFrame(data=d)
  return df

In [ ]:
punctuations = string.punctuation
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

In [ ]:
def spacy_tokenizer(sentence):
    numbers = "0123456789"
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    ## alternative way
    # mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    mytokens_2 = []
    for word in mytokens:
      for char in word:
        if (char in punctuations) or (char in numbers):
          word = word.replace(char, "")
      if word != "":
        mytokens_2.append(word)

    # Return preprocessed list of tokens
    return mytokens_2

In [ ]:
def configs():

    models = list()
    
    # Define config lists
    ngram_range = [(1,1), (1,2), (1, 3), (2, 2), (2, 3), (3, 3)]
    min_df = [1]
    max_df = [1.0]
    analyzer=['word', 'char']
    
    # Create config instances
    for n in ngram_range:
        for i in min_df:
            for j in max_df:
              for a in analyzer:
                    cfg = [n, i, j, a]
                    models.append(cfg)
    return models

configs = configs()
configs[:10]

[[(1, 1), 1, 1.0, 'word'],
 [(1, 1), 1, 1.0, 'char'],
 [(1, 2), 1, 1.0, 'word'],
 [(1, 2), 1, 1.0, 'char'],
 [(1, 3), 1, 1.0, 'word'],
 [(1, 3), 1, 1.0, 'char'],
 [(2, 2), 1, 1.0, 'word'],
 [(2, 2), 1, 1.0, 'char'],
 [(2, 3), 1, 1.0, 'word'],
 [(2, 3), 1, 1.0, 'char']]